In [1]:
#import libraries
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

In [2]:
#access data

data= pd.read_csv('bills.csv')
text = pd.read_csv('billtext.csv', sep='\t')

/Users/Jade/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (11,13,30,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Concatenate the two datasets.  
bills = pd.concat([data, text['bill_text_clean']], axis=1)

In [4]:
data = bills[bills['bill_text_clean']!='Problem']

In [5]:
data= data[['BillID','BillNum','ByReq', 'Commem','Cong','Cosponsr','IntrDate','Mult','MultNo',
           'PassH', 'PassS','PLaw','Private','Title','Veto','Class','ComC','ComR','CumSServ',
            'Gender','LeadCham','LeadSubC','Majority','MRef','NameFirst','NameLast',
          'Party','Postal','bill_text_clean']]

In [6]:
data['PartyName']= data['Party'].map({0.0:'R', 100.0: 'D', 200.0:'R', 328.0:'I'})
data.PartyName.value_counts()

D    7695
R    6791
I     183
Name: PartyName, dtype: int64

In [7]:
data = data[data['PartyName']!='I']

data.dropna(inplace=True)
data.shape

(11573, 30)

In [8]:
#Creates variable to mark bill author as republican or not
data['IsRepublican']= data['PartyName'].map({'R':1, 'D':0})

In [9]:
# removing numbers from bill text
# remove the words "public law" from passed public laws 
import re
def remove_numbers(text):
    nonumbers= re.sub('[^a-zA-Z]', ' ', text)
    return nonumbers.replace('Public Law', ' ')
data['bill_text_nonumbers']= [remove_numbers(x) for x in data.bill_text_clean]

In [10]:
data = data.reset_index()

In [11]:
from sklearn.externals import joblib

In [12]:
#creates a pickle to save my trained logistic regression model for future use
X= data.bill_text_nonumbers
y= data.IsRepublican


vect= CountVectorizer(max_features=5000, stop_words='english')
X_dtm = vect.fit_transform(X)
logreg = LogisticRegression()
logreg.fit(X_dtm, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
joblib.dump(vect, 'rd_vect.pkl')
joblib.dump(logreg, 'rd_class.pkl')

['rd_class.pkl',
 'rd_class.pkl_01.npy',
 'rd_class.pkl_02.npy',
 'rd_class.pkl_03.npy',
 'rd_class.pkl_04.npy']

In [16]:
X= data.bill_text_nonumbers
y= data.PLaw

vect2= TfidfVectorizer(max_features=5000, stop_words='english')
X_dtm = vect2.fit_transform(X)
logreg2= LogisticRegression()
logreg2.fit(X_dtm, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
joblib.dump(vect2, 'pass_vect.pkl')
joblib.dump(logreg2, 'pass_class.pkl')

['pass_class.pkl',
 'pass_class.pkl_01.npy',
 'pass_class.pkl_02.npy',
 'pass_class.pkl_03.npy',
 'pass_class.pkl_04.npy']